In [1]:
from typing import Sequence, TypedDict
from langgraph.graph import StateGraph
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage
import yaml

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "mistral-small:22b-instruct-2409-q6_K",
    #model = "qwen2.5:32b-instruct-q6_K",
    temperature = 0.8,
    num_ctx=8192,
    num_predict = 4098,
)

In [3]:
type Messages = Sequence[AnyMessage]

class Context(TypedDict):
  details: str

class State(TypedDict):
  messages: Sequence[AnyMessage]
  context: Context

pre_sys = SystemMessage(content="You are an agent tasked with providing inspiration to an artist. When the artist describes their request, respond with the first topic derrived from it as context for their continued work. Summarize, rephrase, make conceptual leaps. Do not describe the aritst, only a contextualized, realized topic. You MUST NOT simply repeat their request; you must put it in your own words.")

def pre(state: State) -> State:
  messages = [pre_sys] + state["messages"]
  context = llm.with_structured_output(Context).invoke(messages)

  return { "context": context }

post_sys = SystemMessage(content="You are an artist preparing to embark on a large project. Use the provided context to create a proposal for your project.")

def post(state: State) -> State:
  context = AIMessage(
    content="Context:\n" + yaml.dump(state["context"], sort_keys=False, 
                      default_flow_style=False), 
    label="context")
  messages = [post_sys, context] + state["messages"]

  res = llm.invoke(messages)
  return { "messages": state["messages"] + [res] }


graph_builder = StateGraph(State)
graph_builder.add_node("pre", pre)
graph_builder.add_node("post", post)

graph_builder.add_edge("pre", "post")
graph_builder.set_entry_point("pre")

graph = graph_builder.compile()

output = graph.invoke({ "messages": [HumanMessage(content="I am interested in birds, wind, skies, and evolution.")] })
print(output)
print(output["messages"][-1].content)

{'messages': [HumanMessage(content='I am interested in birds, wind, skies, and evolution.', additional_kwargs={}, response_metadata={}), AIMessage(content=' **Project Proposal: Feathers of Time**\n\n**Artist\'s Statement:**\nMy fascination with birds, wind, skies, and evolution has led me to conceive a multidisciplinary art project titled "Feathers of Time." This project aims to explore the intricate relationship between these elements through an immersive installation that combines sculpture, painting, sound art, and interactive elements.\n\n**Project Description:**\n\n1. **Sculptural Installation - "Winds of Change":**\n   The centerpiece of the project will be a large-scale sculptural installation representing the evolution of birds over time. Utilizing various materials like metal, glass, and resin, I will create a series of stylized bird forms suspended from the ceiling, appearing to soar through the air. Each bird will represent a different stage in avian evolution, from early an